In [ ]:
from ultralytics import YOLO #pretrained yolo models
import cv2 #for image and video processing
import cvzone #for drawing boxes and boundaries text etc
import math
from sort import *

cap=cv2.VideoCapture("../Video/video3.mp4")  #initiating webcam video capturing

model=YOLO("../yolo-weights/yolov8l.pt") #loads yolo model 

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]  # list of class labels

mask= cv2.imread("mask2.png")

trackers= Sort(max_age=30, min_hits=5, iou_threshold=0.3)


while True:  # infinite loop to keep reading frames from the webcam
    success ,img=cap.read()    # reading every frame img holds the frame success is true if reading was succcessful
    imgRegion=cv2.bitwise_and(img,mask)       

    results=model(imgRegion, stream=True) #runs yolo model for that for that the fram read in the last line stream=true allows looping thro detection more efficient 
    
    detection=np.empty((0, 5))
    
    for r in results:
        boxes=r.boxes  #Iterates through detection results ,r.boxes contains all the bounding boxes YOLO found in the frame.
        for box in boxes:  #Loops over each individual detected box.
            #bounding box
            x1,y1,x2,y2=box.xyxy[0]
            x1,y1,x2,y2=int (x1),int (y1),int (x2),int (y2)
            
            w,h=x2-x1,y2-y1
                     # making boxes getting their coordinates changing them to integer then calculate the width and height and makes rectangle with corner using cvzone
            
            
            conf=math.ceil((box.conf[0]*100))/100   #confidence
            
            cls=int(box.cls[0])                   #class name
            currentCLass=classNames[cls]
            if currentCLass=="car" or currentCLass== "motorbike" or currentCLass== "bus" or currentCLass== "truck" and conf>0.5:
            #  cvzone.putTextRect(img,f'{classNames[cls]} {conf}',(max(0,x1),max(35,y1)),offset=3)  
            # draws filled rectantgle with class ane and confidence the rectangle dolesnt go out of the frame
            
             cvzone.cornerRect(img,(x1,y1,w,h),l=15,rt=5)  
             currentArray=np.array([x1,y1,x2,y2,conf])
             detection=np.vstack((detection,currentArray))

        resultsTracker=trackers.update(detection)
        for results in resultsTracker:
            x1,y1,x2,y2,id=results
            x1,y1,x2,y2=int (x1),int (y1),int (x2),int (y2)
            print(results)
            w,h=x2-x1,y2-y1
            cvzone.cornerRect(img,(x1,y1,w,h),l=15,rt=2,colorR=(255,0,0))  
            cvzone.putTextRect(img,f'{int(id)}',(max(0,x1),max(35,y1)),offset=3)


        cv2.imshow("Image", img)
        cv2.imshow("ImageRegion",imgRegion)
    key = cv2.waitKey(0)  # Wait for a key press indefinitely

    if key == ord('q'):   # Quit when 'q' is pressed
        break

cap.release()   #release the webcam
cv2.destroyAllWindows()  #destroy all the windows 


